In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, accuracy_score
from xgboost import XGBClassifier

In [ ]:
class LoanPredictionModel:
    #inisialisasi atribut dan path file
    def __init__(self, file_path):
        self.file_path = file_path
        self.data = None
        self.dataencoded = None
        self.X_train = self.X_test = self.y_train = self.y_test = None
        self.xgb_model = None
        self.catcols = ['person_gender', 'person_education', 'person_home_ownership',
                        'loan_intent', 'previous_loan_defaults_on_file']
        self.numcols = ['person_age', 'person_income', 'person_emp_exp', 'loan_amnt', 'loan_int_rate',
                        'loan_percent_income', 'cb_person_cred_hist_length', 'credit_score']

    #load data dan cek missing value
    def load_data(self):
        self.data = pd.read_csv(self.file_path)
        print("missing value:\n", self.data.isnull().sum())

    #preprocessing data (handling missing value, encode, outlier)
    def preprocess_data(self):
        print("handling missing values...")
        self.data['person_income'].fillna(self.data['person_income'].mean(), inplace=True)
        self.dataencoded = self.data.copy()

        print("encoding categorical columns...")
        le = LabelEncoder()
        for col in self.catcols:
            self.dataencoded[col] = le.fit_transform(self.dataencoded[col])

        print("detecting outliers...")
        for col in self.numcols:
            Q1 = self.dataencoded[col].quantile(0.25)
            Q3 = self.dataencoded[col].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR
            self.dataencoded[col] = np.where(
                self.dataencoded[col] < lower_bound, lower_bound,
                np.where(self.dataencoded[col] > upper_bound, upper_bound, self.dataencoded[col])
            )

        print("removing gender anomalies...")
        self.dataencoded = self.dataencoded[self.dataencoded['person_gender'].isin([0, 1])]

    #split data train dan testing
    def split_data(self):
        print("splitting dataset...")
        X = self.dataencoded.drop(columns=['loan_status'] + self.catcols)
        y = self.dataencoded['loan_status']
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(
            X, y, test_size=0.2, random_state=42
        )

    #hyperparameter tuning dan training model xgb
    def tune_and_train_xgboost(self):
        print("tuning XGBoost model...")
        xgb_params = {
            'n_estimators': [100, 200, 300],
            'max_depth': [3, 6, 9],
            'learning_rate': [0.01, 0.1, 0.2]
        }
        xgb = XGBClassifier(random_state=42)
        xgb_grid = GridSearchCV(xgb, xgb_params, cv=5, scoring='accuracy')
        xgb_grid.fit(self.X_train, self.y_train)
        self.xgb_model = xgb_grid.best_estimator_

        print("best parameters:", xgb_grid.best_params_)

    #evaluasi hasil model
    def evaluate_model(self):
        print("evaluating model...")
        preds = self.xgb_model.predict(self.X_test)
        acc = accuracy_score(self.y_test, preds)
        report = classification_report(self.y_test, preds)
        print("XGBoost performance:")
        print(report)
        print(f"accuracy: {acc}")


In [ ]:
model = LoanPredictionModel("Dataset_A_loan.csv")
model.load_data()

Loading data...
Missing value:
 person_age                           0
person_gender                        0
person_education                     0
person_income                     2250
person_emp_exp                       0
person_home_ownership                0
loan_amnt                            0
loan_intent                          0
loan_int_rate                        0
loan_percent_income                  0
cb_person_cred_hist_length           0
credit_score                         0
previous_loan_defaults_on_file       0
loan_status                          0
dtype: int64


In [ ]:
model.preprocess_data()

Handling missing values...
Encoding categorical columns...
Detecting and capping outliers...
Removing gender anomalies...


<ipython-input-33-2a22c9df2363>:27: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  self.data['person_income'].fillna(self.data['person_income'].mean(), inplace=True)


In [ ]:
model.split_data()

Splitting dataset...


In [ ]:
model.tune_and_train_xgboost()

Tuning XGBoost model...
Best Parameters: {'learning_rate': 0.2, 'max_depth': 3, 'n_estimators': 300}


In [ ]:
model.evaluate_model()

Evaluating model...
XGBoost Performance:
              precision    recall  f1-score   support

           0       0.62      1.00      0.77        10
           1       1.00      0.25      0.40         8

    accuracy                           0.67        18
   macro avg       0.81      0.62      0.58        18
weighted avg       0.79      0.67      0.61        18

Accuracy: 0.6666666666666666
